In [3]:
import numpy as np
import matplotlib.pyplot as plot
import csv
from scipy.optimize import curve_fit
from scipy import integrate
import math

#Import the files and create the lists that will define the x and y coordinates for each datapoint
x = []
y = []
coords = []
file = 'Crater4-5-160kv-1.csv'
path = r"C:\Users\BrendanAlbertHaas\Documents\StardustResearch\TEM\TEM_Spectra\C2113N-A\CSV\V" #V added to end to prevent error with last character being a backslash
filepath = path[:-1] + file #Ignore the V from the previous line

#Start by plotting the data to perform initial sanity checks
with open(filepath, 'r') as csvfile:
    EDSData = csv.reader(csvfile, delimiter=',')
    for row in EDSData:
        x.append(float(row[0]))
        y.append(float(row[1]))
        
#Add option to add another spectrum to the first one
#Define the function:
def AddAnotherSpectrum(InputFile): #assumes same file folder location as previous file
    NewFile = InputFile
    NewFilePath = path[:-1] + NewFile
    
    with open(NewFilePath, 'r') as csvfile:
        EDSData = csv.reader(csvfile, delimiter=',')
        i=0
        for row in EDSData:
            y[i] = y[i] + float(row[1])
            i=i+1

#Now use function to add more spectra as needed to improve stastics as more data is collected
#AddAnotherSpectrum('Crater5-1-1.csv')


#Calculate the background noise
#Average over all counts from 4.0-4.2 KeV, since there should not be any relevant counts in this region of the spectra
NoiseSum = 0
for i in range(420, 441, 1):
    NoiseSum = NoiseSum + y[i]
NoiseAverage = int(round(NoiseSum/21)) #Average the noise per bin, rounding to nearest integer, as our counts per bin will always be integers
#print(NoiseAverage) #Sanity Check


#Remove background noise from spectra
j=0
while j < len(y):
    y[j] = y[j] - NoiseAverage #Subtract average noise from each bin
    if y[j] < 0:
        y[j] = 0 #Negative counts don't make sense, so put the minimum counts per bin at 0
    j=j+1

    
CountsMaxNoAl = max(max(y[21:135]), max(y[165:780])) #Find peak counts, ignoring the early noise the Al/Cu peaks, used for plotting yaxis
#print(CountsMaxNoAl) #sanity check


#Plot the data with the background noise removed  
plot.plot(x,y, color='blue', linestyle='solid')
plot.style.use('seaborn-poster')
plot.xlabel('Energy in KeV', fontsize = 18)
plot.ylabel('Counts', fontsize = 18)
plot.xlim(.2,7.8) #Cutoff plot at 10 KeV, the important peaks are all below 10 KeV
plot.ylim(0,CountsMaxNoAl+500) #Y-max shown in plot will be 500 counts higher than max important peak
plot.title('EDS Spectra from ' + file + '. Please select all significant peaks')

#Add lines to location of potentially present elements
def AddMarkerLine(location, element, colorchoice):
    plot.axvline(x=location, color = colorchoice, linestyle = 'dashed')
    plot.text(location, CountsMaxNoAl+400, element, color=colorchoice) #plot line names near the top (CountsMaxNoAl+400)
AddMarkerLine(1.253, 'Mg', 'green') #Mg K-alpha location
AddMarkerLine(1.486, 'Al', 'green') #Al K-alpha location
AddMarkerLine(1.739, 'Si', 'green') #Si K-alpha location
AddMarkerLine(2.048, 'Pt', 'green') #Pt M location
AddMarkerLine(2.306, 'S', 'green') #S K-alpha location
AddMarkerLine(3.312, 'K', 'green') #K K-alpha location
AddMarkerLine(3.690, 'Ca', 'green') #Ca K-alpha location
AddMarkerLine(6.398, 'Fe', 'green') #Fe K-alpha location
AddMarkerLine(7.471, 'Ni', 'green') #Ni K-alpha location

plot.show()


####################
#We plot our spectrum first
#Then, we click on all the peaks in the plot, and fit that many gaussians.  Helps with many overlapping peaks
fig = plot.figure()
ax = fig.add_subplot(111)
ax.plot(x,y)

#Save the coordinates of each click on the plot, so that we know the rough locations of each peak we wish to analyze
def onclick(event):
    global ix, iy
    ix, iy = float(round(event.xdata,2)), float(round(event.ydata,2)) #Round clicked locations to 2 decimal points
    print('x = %.2f, y = %.2f' % (ix, iy))
    global coords
    coords.append(ix)
    coords.append(iy) #likely unnecessary, but keeping in case y-click coord proves useful later

    #if len(coords) == 2: #A way to exit the plot after a certain number of clicks
    #    fig.canvas.mpl_disconnect(cid)
    #return coords
    
cid = fig.canvas.mpl_connect('button_press_event', onclick)
#print(coords)

NumberOfPeaks=int(len(coords)/2) #halved because y coordinates are redundant
xcoords = coords[::2] #grab x coordinates of the peaks to use as peak center locations
xcoords.sort() #Sort clicked peak locations from left to right


#Define function for Gaussian, which we'll be using to try to fit our data to
def GausFunc(x, a, b, c):
    return a * np.exp((-(x-b)**2)/(2*c**2)) #a = height of curve peak, b = position of center of peak, c = parameter controlling peak width
def TwoGausFunc(x, a1, b1, c1, a2, b2, c2):
    return a1 * np.exp((-(x-b1)**2)/(2*c1**2)) + a2 * np.exp((-(x-b2)**2)/(2*c2**2))
def ThreeGausFunc(x, a1, b1, c1, a2, b2, c2, a3, b3, c3):
    return a1 * np.exp((-(x-b1)**2)/(2*c1**2)) + a2 * np.exp((-(x-b2)**2)/(2*c2**2)) + a3 * np.exp((-(x-b3)**2)/(2*c3**2))


#Define function for checking which element a located peak belongs to
def PeakCheck(PeakCenter):
    if 0.25 <= PeakCenter <= 0.30: return 'C' #PeakCenter ranges are defined by known peak locations for each element
    elif 0.49 <= PeakCenter <= 0.55: return 'O' #Oxygen
    elif 0.68 <= PeakCenter <= 0.74: return 'Fe2' #Iron's L-alpha peak
    #elif 0.90 <= PeakCenter <= 0.97: return 'Cu2' #Copper's L-alpha peak
    #elif 0.97 <= PeakCenter <= 1.04: return 'Zn2' #Zinc's L-alpha peak is very close to Sodium's K-alpha, use discretion here
    elif 1.00 <= PeakCenter <= 1.07: return 'Na' #Sodium
    #elif 1.08 <= PeakCenter <= 1.13: return 'Ga2' #Gallium's L-alpha peak
    elif 1.21 <= PeakCenter <= 1.29: return 'Mg' #Magnesium
    elif 1.45 <= PeakCenter <= 1.53: return 'Al' #...
    elif 1.69 <= PeakCenter <= 1.78: return 'Si' #Not all elements present in this list as some won't be found in cometary material
    elif 2.00 <= PeakCenter <= 2.09: return 'Pt'
    elif 2.26 <= PeakCenter <= 2.35: return 'S'
    elif 2.58 <= PeakCenter <= 2.66: return 'Cl'
    elif 3.27 <= PeakCenter <= 3.35: return 'K'
    elif 3.65 <= PeakCenter <= 3.74: return 'Ca'
    elif 4.47 <= PeakCenter <= 4.55: return 'Ti'
    elif 5.37 <= PeakCenter <= 5.45: return 'Cr'
    elif 5.85 <= PeakCenter <= 5.93: return 'Mn'
    elif 6.35 <= PeakCenter <= 6.44: return 'Fe'
    elif 7.43 <= PeakCenter <= 7.52: return 'Ni'
    elif 8.00 <= PeakCenter <= 8.08: return 'Cu'
    elif 8.59 <= PeakCenter <= 8.68: return 'Zn'
    elif 9.83 <= PeakCenter <= 9.91: return 'Ga'
    else: return 'Unknown'

#Define dictionary to store data: Peak number (left to right) is the key, peak location/counts/element names are list elements tied to dictionary
EleDict = {}

#No need to find exact peak locations, as they will be solved for during the guassian fits
if NumberOfPeaks == 1: #Quick fit of a single selected peak, output the total counts under the peak
    #Best Gaussian fit around the peak selected on the plot
    #curve_fit returns popt (Optimal values for the parameters so that the sum of the squared residuals of f(xdata, *popt) - ydata is minimized)
    #curve_fit also returns pcov (The estimated covariance of popt)
    #First list lower bounds for (a,b,c) together, then list upper bounds for (a,b,c) together
    popt, pcov = curve_fit(GausFunc, x[int(round(xcoords[0]*100,2))-20:int(round(xcoords[0]*100,2))+20], #list x&y ranges to fit over
                           y[int(round(xcoords[0]*100,2))-20:int(round(xcoords[0]*100,2))+20], 
                           maxfev=1000, bounds=((y[int(round(xcoords[0]*100,2))]/2,round(xcoords[0],2)-10,0.01) , #lower bounds
                                                (y[int(round(xcoords[0]*100,2))]*1.2,round(xcoords[0],2)+10,0.10))) #upper bounds
    print(popt)
    Counts = integrate.quad(lambda x: GausFunc(x, popt[0]*100, popt[1], popt[2]),
                        popt[1]-popt[2]*1.412, popt[1]+popt[2]*1.412) #c is related to FWHM*1.2 by a factor of 1.412, counts calculated under FWHM*1.2 width
    print("Peak Location = " + str(round(popt[1],2)))
    print("Total Counts = " + str(Counts[0]))
    #We cannot do Cliff-Lorimer quantitative analysis with only 1 peak, so we'll move on to the two-peak case
    
if NumberOfPeaks == 2: #Quick fit of 2 selected peaks, output the total counts for each peak
    #Best Gaussian fits for the 2 peaks.  If the peaks are not adjacent, may prove inaccurate, 2 separate single-peak fits would be better
    popt, pcov = curve_fit(TwoGausFunc, x[int(round(xcoords[0]*100,2))-20:int(round(xcoords[1]*100,2))+20],
                           y[int(round(xcoords[0]*100,2))-20:int(round(xcoords[1]*100,2))+20], maxfev=1000,
                           bounds=((y[int(round(xcoords[0]*100,2))]/2,round(xcoords[0],2)-10,0.01,y[int(round(xcoords[1]*100,2))]/2,round(xcoords[1],2)-10,0.01),
                                   (y[int(round(xcoords[0]*100,2))]*1.2,round(xcoords[0],2)+10,0.1,y[int(round(xcoords[1]*100,2))]*1.2,round(xcoords[1],2)+10,0.1)))
    print(popt)
    Counts1 = integrate.quad(lambda x: GausFunc(x, popt[0]*100, popt[1], popt[2]),
                        popt[1]-popt[2]*1.412, popt[1]+popt[2]*1.412)
    Counts2 = integrate.quad(lambda x: GausFunc(x, popt[3]*100, popt[4], popt[5]),
                        popt[4]-popt[5]*1.412, popt[4]+popt[5]*1.412)
    print("Peak Location 1 = " + str(round(popt[1],2)))
    print("Peak 1 Total Counts = " + str(Counts1[0]))
    print("Peak Location 2 = " + str(round(popt[4],2)))
    print("Peak 2 Total Counts = " + str(Counts2[0]))
    
    #Fill dictionary with keys and data
    EleDict['1'] = [math.ceil(popt[1]*100)/100, math.ceil(Counts1[0])] #Round up to 2 digits
    EleDict['2'] = [math.ceil(popt[4]*100)/100, math.ceil(Counts2[0])]
    EleDict['1'].append(PeakCheck(EleDict['1'][0])) #add the element the peak belongs to to the dictionary listing
    EleDict['2'].append(PeakCheck(EleDict['2'][0]))
    print(EleDict)
    
if NumberOfPeaks == 3: #Quick fit of 3 selected peaks, will use this as a base to establish a system for quantifying 3+ peaks
    #Best Gaussian fits for the 3 peaks.  If the peaks are not adjacent, may prove inaccurate, separate peak fits would be better
    popt, pcov = curve_fit(ThreeGausFunc, x[int(round(xcoords[0]*100,2))-20:int(round(xcoords[2]*100,2))+20], 
                           y[int(round(xcoords[0]*100,2))-20:int(round(xcoords[2]*100,2))+20], maxfev=1000,
                           bounds=((y[int(round(xcoords[0]*100,2))]/2,round(xcoords[0],2)-10,0.01,
                                    y[int(round(xcoords[1]*100,2))]/2,round(xcoords[1],2)-10,0.01,
                                    y[int(round(xcoords[2]*100,2))]/2,round(xcoords[2],2)-10,0.01),
                                   (y[int(round(xcoords[0]*100,2))]*1.2,round(xcoords[0],2)+10,0.10,
                                    y[int(round(xcoords[1]*100,2))]*1.2,round(xcoords[1],2)+10,0.10,
                                    y[int(round(xcoords[2]*100,2))]*1.2,round(xcoords[2],2)+10,0.10)))
    print(popt)
    Counts1 = integrate.quad(lambda x: GausFunc(x, popt[0]*100, popt[1], popt[2]),
                        popt[1]-popt[2]*1.412, popt[1]+popt[2]*1.412)
    Counts2 = integrate.quad(lambda x: GausFunc(x, popt[3]*100, popt[4], popt[5]),
                        popt[4]-popt[5]*1.412, popt[4]+popt[5]*1.412)
    Counts3 = integrate.quad(lambda x: GausFunc(x, popt[6]*100, popt[7], popt[8]),
                        popt[7]-popt[8]*1.412, popt[7]+popt[8]*1.412)
    print("Peak Location 1 = " + str(round(popt[1],2)))
    print("Peak 1 Total Counts = " + str(Counts1[0]))
    print("Peak Location 2 = " + str(round(popt[4],2)))
    print("Peak 2 Total Counts = " + str(Counts2[0]))
    print("Peak Location 3 = " + str(round(popt[7],2)))
    print("Peak 3 Total Counts = " + str(Counts3[0]))
    
    #Fill dictionary with keys and data
    EleDict['1'] = [math.ceil(popt[1]*100)/100, math.ceil(Counts1[0])] #Round up to 2 digits
    EleDict['2'] = [math.ceil(popt[4]*100)/100, math.ceil(Counts2[0])]
    EleDict['3'] = [math.ceil(popt[7]*100)/100, math.ceil(Counts3[0])]
    EleDict['1'].append(PeakCheck(EleDict['1'][0])) #add the element the peak belongs to to the dictionary listing
    EleDict['2'].append(PeakCheck(EleDict['2'][0]))
    EleDict['3'].append(PeakCheck(EleDict['3'][0]))
    print(EleDict)
    
if NumberOfPeaks > 25:
    print("Too many peaks selected, calm down!") #In case of absent-minded clicking of the plot
    
#CurveFitParameters = []
if 3 < NumberOfPeaks <=25: #Most spectra will likely have many peaks present, we will need to interatively solve for each one from left to right
    #For the first peak, we will do a Gaussian fit of the 2 leftmost peaks, use the fit for the first peak, but disregard the second one for now
    #2nd peak will be fit in next step
    popt, pcov = curve_fit(TwoGausFunc, x[int(round(xcoords[0]*100,2))-10:int(round(xcoords[1]*100,2))+10],
                        y[int(round(xcoords[0]*100,2))-10:int(round(xcoords[1]*100,2))+10], maxfev=10000,
                        bounds=((y[int(round(xcoords[0]*100,2))]/2,round(xcoords[0],2)-10,0.03,y[int(round(xcoords[1]*100,2))]/2,round(xcoords[1],2)-10,0.03),
                                (y[int(round(xcoords[0]*100,2))]*1.2,round(xcoords[0],2)+10,0.12,y[int(round(xcoords[1]*100,2))]*1.2,round(xcoords[1],2)+10,0.12)))
        
    Counts1 = integrate.quad(lambda x: GausFunc(x, popt[0]*100, popt[1], popt[2]),
                    popt[1]-popt[2]*1.412, popt[1]+popt[2]*1.412)
    print("Peak Location 1 = " + str(round(popt[1],2)))
    print("Peak 1 Total Counts = " + str(int(Counts1[0])))
    PreviousPeakParameters = []
    PreviousPeakParameters.append(popt[0])
    PreviousPeakParameters.append(popt[1])
    PreviousPeakParameters.append(popt[2])
    #print(PreviousPeakParameters)
    EleDict['1'] = [math.ceil(popt[1]*100)/100, math.ceil(Counts1[0])] #Add counts data to our dictionary
    EleDict['1'].append(PeakCheck(EleDict['1'][0]))
    
    #2nd peak to 2nd to last peak will be found in this loop
    for Peak in range(0, len(xcoords)-2): #Last peak not included yet, it will be handled separately
        #print(Peak)
        #print(PreviousPeakParameters)
        #Analyze 3 peaks at a time; leftmost peak will have been already fitted from previous fitting.
        #Middle peak will have its values saved, right one will be discarded for now, but fit in the same way next step
        popt, pcov = curve_fit(ThreeGausFunc, x[int(round(xcoords[Peak]*100,2))-10:int(round(xcoords[Peak+2]*100,2))+10], 
                           y[int(round(xcoords[Peak]*100,2))-10:int(round(xcoords[Peak+2]*100,2))+10], maxfev=10000,
                           bounds=((PreviousPeakParameters[0],PreviousPeakParameters[1],PreviousPeakParameters[2], #parameters already found on previous peak search
                                    y[int(round(xcoords[Peak+1]*100,2))]/2,round(xcoords[Peak+1],2)-10,0.03,
                                    y[int(round(xcoords[Peak+2]*100,2))]/2,round(xcoords[Peak+2],2)-10,0.03),
                                   (PreviousPeakParameters[0]+1,PreviousPeakParameters[1]+.01,PreviousPeakParameters[2]+.001, #parameters already found on previous peak search
                                    y[int(round(xcoords[Peak+1]*100,2))]*1.2,round(xcoords[Peak+1],2)+10,0.12,
                                    y[int(round(xcoords[Peak+2]*100,2))]*1.2,round(xcoords[Peak+2],2)+10,0.12)))
        
        Counts1 = integrate.quad(lambda x: GausFunc(x, popt[3]*100, popt[4], popt[5]),
                    popt[4]-popt[5]*1.412, popt[4]+popt[5]*1.412)
        print("Peak Location " + str(Peak+2) + " = " + str(round(popt[4],2)))
        print("Peak " + str(Peak+2) + " Total Counts = " + str(int(Counts1[0])))
        
        #Replace initial peak with parameters we discovered here
        PreviousPeakParameters=[]
        PreviousPeakParameters.append(popt[3])
        PreviousPeakParameters.append(popt[4])
        PreviousPeakParameters.append(popt[5])
        EleDict[str(Peak+2)] = [math.ceil(popt[4]*100)/100, math.ceil(Counts1[0])] #Add counts data to our dictionary
        EleDict[str(Peak+2)].append(PeakCheck(EleDict[str(Peak+2)][0]))
        
    print(PreviousPeakParameters)
    #Now analyze the final peak in the same way that we analyzed the first peak
    popt, pcov = curve_fit(TwoGausFunc, x[int(round(xcoords[-2]*100,2))-10:int(round(xcoords[-1]*100,2))+10],
                        y[int(round(xcoords[-2]*100,2))-10:int(round(xcoords[-1]*100,2))+10], maxfev=10000,
                        bounds=((y[int(round(xcoords[-2]*100,2))]/2,round(xcoords[-2],2)-10,0.03,y[int(round(xcoords[-1]*100,2))]/2,round(xcoords[-1],2)-10,0.03),
                                (y[int(round(xcoords[-2]*100,2))]*1.2,round(xcoords[-2],2)+10,0.12,y[int(round(xcoords[-1]*100,2))]*1.2,round(xcoords[-1],2)+10,0.12)))
    
    Counts2 = integrate.quad(lambda x: GausFunc(x, popt[3]*100, popt[4], popt[5]),
                    popt[4]-popt[5]*1.412, popt[4]+popt[5]*1.412)
    print("Peak Location " + str(len(xcoords)) + " = " + str(round(popt[4],2)))
    print("Peak " + str(len(xcoords)) + " Total Counts = " + str(int(Counts2[0])))
    
    PreviousPeakParameters=[]
    PreviousPeakParameters.append(popt[3])
    PreviousPeakParameters.append(popt[4])
    PreviousPeakParameters.append(popt[5])
    print(PreviousPeakParameters)
    EleDict[str(len(xcoords))] = [math.ceil(popt[4]*100)/100, math.ceil(Counts2[0])] #Add counts data to our dictionary
    EleDict[str(len(xcoords))].append(PeakCheck(EleDict[str(len(xcoords))][0]))
    print(EleDict)

#So we now have a dictionary (EleDict) that has all the peaks numbered as keys
#The peak's location, counts under the 1.2*FWHM, and peak's elemental source are the dictionary's entries
#We can use the latter 2 to perform Cliff-Lorimer analysis, and quantify the composition of the sample

x = 1.48, y = 1451.73
x = 1.74, y = 1098.21
x = 1.02, y = 1447.89
[  1.27871412e+03   1.05111907e+00   1.00000000e-01   2.06850822e+04
   1.48377711e+00   4.95097342e-02   1.00534102e+03   1.73445181e+00
   6.22452775e-02]
Peak Location 1 = 1.05
Peak 1 Total Counts = 26989.901181891993
Peak Location 2 = 1.48
Peak 2 Total Counts = 216160.17379410143
Peak Location 3 = 1.73
Peak 3 Total Counts = 13208.321876486389
{'1': [1.06, 26990, 'Na'], '2': [1.49, 216161, 'Al'], '3': [1.74, 13209, 'Si']}


In [5]:
#Part 2:  Cliff-Lorimer Analysis
#The weight percents of each element Ca and Cb are related to the characteristic intensities Ia and Ib:
# Ca/Cb = Kab * (Ia/Ib)
#We can also assume Ca + Cb = 1, as the total must be 100% of the composition of the material
#Adding a 3rd element simply adds another equation to the system: Cb/Cc = Kbc * (Ib/Ic), now with Ca+Cb+Cc=1
#Also note, K factors are related thus: Kab = Kac/Kbc, or Kab = 1/Kba
#Our I values will be the counts we have determined in the first part of this code.
#K factors use Si as the baseline for comparison, and are known from previous measurements of standards
#Now, we solve for the C factors using the known I and K values and our system of equations

from tkinter import *
import numpy as np

#We normalize the count rates to Si; each element has a constant compared to SiCounts
#KSi=   1.0  #Standard of comparison with other elements (has strongest signal in SEM/TEM, so good for comparisons)
#KMgSi= 1.19 #Value from Kevin, Value from NSS software is 1.62,  #Theory = 1.37, another paper said 1.92
#KFeSi= 1.87 #Value from Kevin, Theory = 1.07, another paper said 1.057
#KSSi = 1.27 #Value from Kevin, Theory = 1.02, another paper said 0.93
#KCaSi= 1.27 #Value from Kevin, Value from NSS software=0.35  #Theory = 0.90, another paper said .915
#KNiSi= 1.95 #Value from Kevin, Theory = 1.13, another paper said 1.102 
#KAlSi= 1.03 #Value from Kevin, Value from NSS software=1.83  #Theory = 1.16, another paper said 1.29
#KCrSi= 1.68 #Value from Kevin, Theory = 1.01, another paper said .976
#KMnSi= 1.23 #Value from Kevin, Theory = 1.06, another paper said 1.03

#My K Factors:
#KSi = 1.0 KMgSi = 0.936 KFeSi = 2.055 KCaSi = 0.835 KOSi  = 15.87 KSSi  = 1.55 KAlSi = 0.94

KFactorDict = {'Si':1, 'Fe':2.06, 'Ca': 0.84, 'O': 15.9, 'S':1.55, 'Mg':0.94, 'S':1.55, 'Al':0.94} #Factors relative to Si (KFeSi, for ex.)

#Atomic weights for later conversion from weight % to atomic %
#WtSi = 28.084 WtMg = 24.30 WtFe = 55.845 WtS  = 32.06 WtCa = 40.078 WtNi = 58.693 WtAl = 26.982 WtCr = 51.996 WtMn = 54.938

AtomicWtDict = {'Si':28.084,'Mg':24.30,'Fe':55.845,'S':32.06,'Ca':40.078,'Ni':58.693,'Al':26.982,'Cr':51.996,'Mn':54.938}

def PeaksWanted(InputDict): #Take the dictionary of peaks/elements from the first part of code
    global ElementList 
    ElementList = []
    for key in InputDict:
        ElementList.append(InputDict[key][2])
    print("Elements present in original spectra:" + str(ElementList))
    
    
    #Ask for manual input of which elements we'd like to quantify with Cliff Lorimer analysis
    root = Tk()

    ElementPresent=[]
    for Ele in range(0, len(ElementList)):
        ElementPresent.append(0) #Initialize with no elements present
    
    #ElementPresent=[0,0,0,0,0,0,0,0,0,0,0] #Initialize with no elements present
    Label(root, text="Check the elements you want to quantify").grid(row=0, sticky=W)
    for Count,Element in enumerate(ElementList):
        ElementPresent[Count] = IntVar()
        Checkbutton(root, text=str(Element) + "?", variable=ElementPresent[Count]).grid(row=Count+1, sticky=W)
    
    Button(root, text='Quit', command=root.destroy).grid(row=len(ElementList)+2, sticky=W, pady=4)
    root.mainloop()
    
    ElePres=[]
    for Count, Ele in enumerate(ElementPresent):
        #print(ElementPresent[Count].get())
        ElePres.append(ElementPresent[Count].get())
    return(ElePres)
    #print(ElePres)
    
PeaksToMeasure = PeaksWanted(EleDict)
#print(PeaksToMeasure)
    #So we have now have 3 bundles of variables:
    #1:  EleDict from part 1, which still has the peaks linked to the counts
    #2:  EleList, a list of each element present EX: EleList=['Pt','S','Fe','O']
    #3: ElePres, a corresponding list of peaks we want EX: ElePres=[1,0,1,1]
    #In this example, we want Pt, Fe, and O; but not S, which is labeled 0

#Simplify results thus far, create a list alternating between Elements we want to measure and their counts
ElementsAndCounts = []
for Count in range(0,len(PeaksToMeasure)):
    if PeaksToMeasure[Count] == 1:
        ElementsAndCounts.append(ElementList[Count])
        for Key in EleDict:
            if EleDict[Key][2] == ElementList[Count]:
                ElementsAndCounts.append(EleDict[Key][1])
print("Elements to quantify and counts per element: " + str(ElementsAndCounts))

#So we now have a list of the elements we are going to be quantifying, as well as their counts

def CliffLorimer(InputList):
    NumberOfElements = int(len(InputList)/2)
    
    if NumberOfElements == 1:
        print("Cliff-Lorimer analysis cannot be performed on a single element!")
    
    if NumberOfElements == 2: #For the 2 element case
        #2 equations to solve: Ca/Cb=Kab * Ia/Ib, and Ca+Cb = 1; Ca and Cb are the unknowns
        #For Fe and S, for example, we have CFe/CS = KFeS * IFe/IS
        Ia = InputList[1]
        Ib = InputList[3]
        
        #Now we need the relevant K factor; if the two elements are Fe and S, we need KFeS; KFeS = KFeSi/KSSi
        #In other words, we'd need KFeSi and KSSi; 'Fe' and 'S' are the 1st and 3rd elements in our InputList in this example
        #So we need a dictionary of Si-linked K-factors that includes KFeSi and KSSi and to search it for these values
        for element in KFactorDict:
            if InputList[0] == element:
                Ka = KFactorDict[element]
            if InputList[2] == element:
                Kb = KFactorDict[element]
        
        #So for our example, we'd now have Ka = KSSi and kb = KFeSi
        #We need KSFe, which is KSSi/KFeSi, so the K factor we need is Ka/Kb
        Kab = Ka/Kb
        
        #So we have Ia, Ib, and Kab.  We need to solve for Ca and Cb.  Turn these into matricies
        MatrixA = np.matrix([[1,-Kab * Ia/Ib],[1,1]])
        MatrixB = np.matrix([[0],[1]])
        MatrixAInverse = np.linalg.inv(MatrixA)
        
        MatrixX = MatrixAInverse*MatrixB
        #print(MatrixX)
        
        #So we have the weight percentages
        Ca = MatrixX.item(0)
        Cb = MatrixX.item(1)
        
        print("Weight Percentages: " + str(InputList[0]) + ": " + str(Ca) + ", " + str(InputList[2]) + ": " + str(Cb))
        
        #Now we want to solve for atomic percentages
        for element in AtomicWtDict:
            if InputList[0] == element:
                Wta = AtomicWtDict[element]
            if InputList[2] == element:
                Wtb = AtomicWtDict[element]
                
        Ma = Ca/Wta
        Mb = Cb/Wtb
        MTotal = Ma + Mb
        CaAtomic = Ma/MTotal
        CbAtomic = Mb/MTotal
        
        print("Atomic Percentages: " + str(InputList[0]) + ": " + str(CaAtomic) + ", " + str(InputList[2]) + ": " + str(CbAtomic))
        
    #Solve in the case of 3 elements    
    if NumberOfElements == 3:
        Ia = InputList[1]
        Ib = InputList[3]
        Ic = InputList[5]
        
        for element in KFactorDict:
            if InputList[0] == element:
                Ka = KFactorDict[element]
            if InputList[2] == element:
                Kb = KFactorDict[element]
            if InputList[4] == element:
                Kc = KFactorDict[element]
                
        Kab = Ka/Kb
        Kbc = Kb/Kc
        
        MatrixA = np.matrix([[1,-Kab * Ia/Ib,0],[0,1,-Kbc*Ib/Ic],[1,1,1]])
        MatrixB = np.matrix([[0],[0],[1]])
        MatrixAInverse = np.linalg.inv(MatrixA)
        
        MatrixX = MatrixAInverse*MatrixB
        
        Ca = MatrixX.item(0)
        Cb = MatrixX.item(1)
        Cc = MatrixX.item(2)
    
        print("Weight Percentages: " + str(InputList[0]) + ": " + str(Ca) + ", " + str(InputList[2]) + ": " + 
              str(Cb) + ", " + str(InputList[4]) + ": " + str(Cc))
        
        for element in AtomicWtDict:
            if InputList[0] == element:
                Wta = AtomicWtDict[element]
            if InputList[2] == element:
                Wtb = AtomicWtDict[element]
            if InputList[4] == element:
                Wtc = AtomicWtDict[element]
        
        Ma = Ca/Wta
        Mb = Cb/Wtb
        Mc = Cc/Wtc
        MTotal = Ma + Mb + Mc
        CaAtomic = Ma/MTotal
        CbAtomic = Mb/MTotal
        CcAtomic = Mc/MTotal
        
        print("Atomic Percentages: " + str(InputList[0]) + ": " + str(CaAtomic) + ", " + str(InputList[2]) + ": " + 
              str(CbAtomic) + ", " + str(InputList[4]) + ": " + str(CcAtomic))
        
    #Try to apply patterns from previous cases to apply in all cases of 4+ elements
    if NumberOfElements >= 4:
        IFactors = []
        for i in range (1,len(InputList),2): #Grab every other value
            IFactors.append(InputList[i]) #List of 4 I values (Ia through Id) in order
        #print(IFactors)
        
        KList = [] #Grab needed K factors relative to Si
        for i in range (0, len(InputList),2):
            for element in KFactorDict:
                if InputList[i] == element:
                    KList.append(KFactorDict[element]) #List of initial K factors(Ka through Kd) in order
        #print(KList)
        
        NewKList = [] #Produce new K factors relative to the selected elements for analysis
        for i in range (0,NumberOfElements-1): #Need one less factor; Ka through Kd gives Kab, Kbc, and Kcd (3 factors), for example
            NewKFactor = KList[i]/KList[i+1]
            NewKList.append(NewKFactor)
        #print(NewKList)
            
        #Now create the matricies to solve, as we have the I values and the K factors stored in lists
        #Create an empty matrix first, of the appropriate size
        MatrixA = np.zeros((NumberOfElements,NumberOfElements))
        MatrixB = np.zeros((NumberOfElements,1))
        
        MatrixB[NumberOfElements-1][0]=1 #Only the last value of Matrix B is nonzero, and it's equal to 1
        
        #Put appropriate values into appropriate locations in MatrixA
        for i in range (0,NumberOfElements): #Set bottom row of matrix to all 1's
            MatrixA[NumberOfElements-1][i] = 1 
        
        for i in range (0,NumberOfElements): #Set diagonal to all 1's
            MatrixA[i][i] = 1
            
        for i in range (0,NumberOfElements-1): #Set K/I factors; 1 less of these terms (like NewKList)
            MatrixA[i][i+1]=NewKList[i]*(-1)*IFactors[i]/IFactors[i+1]
        
        #print(MatrixA)
        #print(MatrixB)
        
        MatrixX = np.linalg.solve(MatrixA,MatrixB)
        #print(MatrixX)
        
        #Create list of calculated C factors
        CFactors = []
        for i in range (0,NumberOfElements):
            CFactors.append(MatrixX[i][0])
        #print(CFactors)
            
        for i in range (0,NumberOfElements):
            print("Weight percent of " + str(InputList[i*2]) + ": " + str(CFactors[i]))
        
        #Now calculate atomic percent for each element
        AtomicWtList = []
        for i in range (0, len(InputList),2): 
            for element in AtomicWtDict:
                if InputList[i] == element:
                    AtomicWtList.append(AtomicWtDict[element])
        #print(AtomicWtList)
                    
        #Calculate M factors
        MFactors = []
        MTotal = 0
        for i in range (0, len(CFactors)):
            MFactors.append(CFactors[i]/AtomicWtList[i])
            MTotal = MTotal + (CFactors[i]/AtomicWtList[i])
        #print(MFactors)
        #print(MTotal)
        
        AtomicResults = []
        for i in range (0, len(MFactors)):
            AtomicResults.append(MFactors[i]/MTotal)
            print("Atomic percent of " + str(InputList[i*2]) + ": " + str(AtomicResults[i]))
            
        #Calculate and print atomic percent for each element
            
CliffLorimer(ElementsAndCounts)

Elements present in original spectra:['Na', 'Al', 'Si']
Elements to quantify and counts per element: ['Al', 216161, 'Si', 13209]
Weight Percentages: Al: 0.9389603546833613, Si: 0.061039645316638594
Atomic Percentages: Al: 0.9412147150463903, Si: 0.05878528495360974
